Pleaes run preprocess_Data module first before run this module.

Author: xzhang867

## Import Packages

In [1]:
import pandas as pd

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

from google.colab import drive

## Map Google Drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


## Build CNN Model

### Set Common Parameters



In [3]:
nb_train_samples =2312
nb_validation_samples = 663
epochs = 10
img_width, img_height = 256, 256
batch_size = 16

### Generate Training Dataset

In [4]:
train_data_dir = '/content/drive/MyDrive/practicum/data_image_classification/train'

train_datagen = ImageDataGenerator(
	rescale=1. / 255,
	shear_range=0.2,
	zoom_range=0.2,
	horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
	train_data_dir,
	target_size=(img_width, img_height),
	batch_size=batch_size,
	class_mode='categorical')

Found 2312 images belonging to 5 classes.


### Generate Validation Dataset

In [5]:
validation_data_dir = '/content/drive/MyDrive/practicum/data_image_classification/validation'

validation_datagen = ImageDataGenerator(rescale=1. / 255)

validation_generator = validation_datagen.flow_from_directory(
	validation_data_dir,
	target_size=(img_width, img_height),
	batch_size=batch_size,
	class_mode='categorical')

Found 663 images belonging to 5 classes.


### Generate Test Dataset

In [6]:
test_data_dir = '/content/drive/MyDrive/practicum/data_image_classification/test'

test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
	test_data_dir,
	target_size=(img_width, img_height),
	batch_size=batch_size,
	class_mode='categorical')

Found 328 images belonging to 5 classes.


### Prepare CNN Model

In [7]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

# initiate CNN model
model = Sequential()
model.add(Conv2D(32, (2, 2), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# add layer
model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# add layer
model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# add layer
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))

# add dropout to avoid overfitting
model.add(Dropout(0.5))

# output layer contains 5 neuron
model.add(Dense(5))
model.add(Activation('softmax'))

### Train the Model

In [8]:
model.compile(loss='categorical_crossentropy',
			optimizer='rmsprop',
			metrics=['accuracy'])

In [9]:
model.fit(
	train_generator,
	steps_per_epoch=nb_train_samples // batch_size,
	epochs=epochs,
	validation_data=validation_generator,
	validation_steps=nb_validation_samples // batch_size)

Epoch 1/10
144/144 [==============================] - 686s 5s/step - loss: 1.4013 - accuracy: 0.4325 - val_loss: 1.0910 - val_accuracy: 0.5930
Epoch 2/10
144/144 [==============================] - 172s 1s/step - loss: 1.1219 - accuracy: 0.5592 - val_loss: 0.8911 - val_accuracy: 0.6357
Epoch 3/10
144/144 [==============================] - 172s 1s/step - loss: 0.9853 - accuracy: 0.6189 - val_loss: 0.8208 - val_accuracy: 0.6784
Epoch 4/10
144/144 [==============================] - 172s 1s/step - loss: 0.9106 - accuracy: 0.6590 - val_loss: 0.6574 - val_accuracy: 0.7607
Epoch 5/10
144/144 [==============================] - 172s 1s/step - loss: 0.8372 - accuracy: 0.6725 - val_loss: 0.7072 - val_accuracy: 0.7348
Epoch 6/10
144/144 [==============================] - 172s 1s/step - loss: 0.7996 - accuracy: 0.6929 - val_loss: 0.7338 - val_accuracy: 0.7241
Epoch 7/10
144/144 [==============================] - 172s 1s/step - loss: 0.7783 - accuracy: 0.7017 - val_loss: 0.7524 - val_accuracy: 0.7119

## Evaluate Model (Test Data)

In [10]:
evaluate = model.evaluate(test_generator)
print(evaluate)

21/21 [==============================] - 82s 4s/step - loss: 0.6150 - accuracy: 0.7713
[0.6150439381599426, 0.7713414430618286]


## Save Model

In [11]:
model.save('model_saved.h5')